## Google Maps Scrapping

In [ ]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import csv

# Setup WebDriver
driver = webdriver.Chrome()

# URL of the web page
url = "https://www.google.com/maps/search/cafeterias+cerca+de+Flores,+Ciudad+Aut%C3%B3noma+de+Buenos+Aires/@-34.6301648,-58.476723,14z/data=!3m1!4b1?entry=ttu"

# Open the web page
driver.get(url)

# Set an implicit wait time to wait for JavaScript to render
driver.implicitly_wait(30)  # Wait for max 30 seconds

def scroll_panel_with_page_down(driver, panel_xpath, presses, pause_time):
    """
    Scrolls within a specific panel by simulating Page Down key presses.

    :param driver: The Selenium WebDriver instance.
    :param panel_xpath: The XPath to the panel element.
    :param presses: The number of times to press the Page Down key.
    :param pause_time: Time to pause between key presses, in seconds.
    """
    # Find the panel element
    panel_element = driver.find_element(By.XPATH, panel_xpath)
    
    # Ensure the panel is in focus by clicking on it
    actions = ActionChains(driver)
    actions.move_to_element(panel_element).perform()

    # Send the Page Down key to the panel element
    for _ in range(presses):
        actions = ActionChains(driver)
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(pause_time)

# Scroll the panel
panel_xpath = "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div"
scroll_panel_with_page_down(driver, panel_xpath, presses=20, pause_time=2)

# Get the page HTML source
page_source = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Find all elements using its class
titles = soup.find_all('a', class_="hfpxzc")
ratings = soup.find_all('span', class_='MW4etd')
reviews = soup.find_all('span', class_='UY7F9')
services = soup.find_all('div', class_='Ahnjwc')

# Print the number of places found
elements_count = len(titles)
print(f"Number of places found: {elements_count}")

# Specify the CSV file path
csv_file_path = 'places.csv'

# Open a CSV file in write mode
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)
    
    # Write the header row (optional, adjust according to your data)
    csv_writer.writerow(['Place', 'Rating', 'Reviews', 'Service options'])
    
    # Write the extracted data
    for i, title in enumerate(titles):
        title_text = title.get('aria-label', 'N/A')
        rating_text = (ratings[i].text + "/5") if i < len(ratings) else 'N/A' # Ensure we have a rating and reviews for each title, defaulting to 'N/A' if not found
        review_count_text = reviews[i].text if i < len(reviews) else 'N/A'
        service_text = services[i].text if i < len(services) else 'N/A'

        # Write a row to the CSV file
        if title_text:
            csv_writer.writerow([title_text, rating_text, review_count_text, service_text])

print(f"Data has been saved to '{csv_file_path}'")

# Close the WebDriver
driver.quit()
